In [1]:
import matplotlib.pyplot as plt 
from matplotlib.gridspec import GridSpec
import matplotlib
import pandas as pd 
import numpy as np 
import os
import shutil

In [2]:
# создание и удаление директорий  
def create_dir(path: str):
    if os.path.isdir(path):
        shutil.rmtree(path, ignore_errors=True)
        os.mkdir(path)
    else:
        os.mkdir(path)

# нахождение всех вложенных директорий 
def find_directories_data_folder(data_folder: str) -> list:
    dir_list = []
    for i in os.listdir(data_folder):
        if os.path.isdir(f'{data_folder}/{i}'):
            dir_list.append(f'{i}')
    return dir_list

# нахождение всех вложенных файлов
def find_files_in_data_folder(data_folder: str) -> list:
    file_list = []
    for i in os.listdir(data_folder):
        if os.path.isfile(f'{data_folder}/{i}'):
            file_list.append(f'{i}')
    return file_list

# нахождение всех файлов в директориях
def find_files_in_directories(data_folder: str) -> list:
    dir_list = find_directories_data_folder(data_folder=data_folder)
    dir_and_files_dict = []
    for i in dir_list:
        onlyfiles = [int(f.replace('.data','')) for f in os.listdir(os.path.abspath(data_folder + '\\' +  i)) if os.path.isfile(os.path.join(data_folder +'\\' +   i, f))]
        dir_and_files_dict.append(onlyfiles)
    return dir_and_files_dict

# составление списка header файлов по имющимся файлам в папке
def find_header(data_folder: str)-> list:
    dir_list = find_directories_data_folder(data_folder=data_folder)
    files_list = find_files_in_data_folder(data_folder=data_folder)
    header_list = []
    for i in dir_list:
        if f'{i}.header' in files_list:
            header_list.append(f'{i}.header')
    return header_list

# считываем данные об измерениях из header файлов
def get_all_measurments_info(data_folder)-> dict:
    header_list = find_header(data_folder=data_folder)
    splite_line = '______________________________________________________'
    all_measurments_info = {}
    for i in header_list:
        with open(f'{data_folder}\{i}') as f:
            header_lines = []
            for k in f.readlines()[:-2]:
                header_lines.append(k.replace('\n', ''))
            single_contact_info = {}
            for j in range(len(header_lines)):
                if header_lines[j] == splite_line:
                    number, *m_type = header_lines[j + 3].split()
                    file_path = header_lines[j + 8].split()[-1]
                    single_contact_info[int(number)] = {'type': ' '.join(m_type), 'file_path': file_path}
                else:
                    continue
        all_measurments_info[i.replace('.header', '')] =  single_contact_info
    return all_measurments_info

# получение дискретной палитры цветов
def get_colors_from_cmap(cmap_name: str, lenght: int) -> list:
    cmap = plt.get_cmap(cmap_name)
    individual_colors = [cmap(i / lenght) for i in range(lenght)]
    return individual_colors

# поэтапное рисование графика 
def draw_line_by_step(ax, data_V, data_I):
    for i in range(len(data_V)-1):
        ax.plot([data_V[i], data_V[i+1]], np.abs([data_I[i], data_I[i+1]]), solid_capstyle='round')

# раскашивание линий в градиент
def colorize_line(ax: matplotlib.axes, colormap_name: str, lenght: int):
    colors = get_colors_from_cmap(colormap_name, lenght)
    lines = ax.get_lines()
    for i in range(lenght-1):
        lines[i].set_color(colors[i])

# получение данных с указанных фалов и директорий
def get_single_data_from_path(data_folder: str, file_path: str) -> pd.DataFrame:
    single_data_path = os.path.abspath(data_folder + '\\' + file_path)
    single_df = pd.read_csv(single_data_path, delimiter='   ', skiprows=16, engine='python', header=None, encoding='ISO-8859-1').astype(np.float32)
    single_df.rename(columns = {0: 'voltage', 1: 'current', 2: 'resistance'}, inplace=True)
    return single_df

# рисование одного графикаи змерения типа DC IV 
def draw_single_DC_IV(data_V: pd.Series, data_I: pd.Series, fig: matplotlib.figure):
    gs = GridSpec(ncols=26, nrows=1, figure=fig)
    ax = fig.add_subplot(gs[:25])
    ax_1 = fig.add_subplot(gs[-1])
    data_len = len(data_V)
    draw_line_by_step(ax, data_V, data_I)
    colorize_line(ax, 'inferno', data_len)
    ax.set_yscale('log')
    ax.grid(which='major', linewidth = 0.6)
    ax.grid(which='minor', linewidth = 0.2)
    sm = plt.cm.ScalarMappable(cmap=plt.cm.inferno, norm=plt.Normalize(vmin=0, vmax=data_len))
    cbar = fig.colorbar(sm, cax= ax_1)
    cbar.set_ticks([0, data_len])
    cbar.set_ticklabels(['start','end'], size = 15)


def draw_all_DC_IV_from_data_folder(data_folder: str):
    all_info = get_all_measurments_info(data_folder=data_folder)
    global_graph_folder = '_'.join([data_folder, 'graphs'])
    create_dir(global_graph_folder)
    for i in list((all_info.keys())):
        local_graph_folder = '\\'.join([global_graph_folder, i]) 
        create_dir(local_graph_folder)
        for j in list(all_info[i].keys()):
            if all_info[i][j]['type'] == 'DC IV':
                fig = plt.figure(figsize=[10, 5], constrained_layout=True)
                data = get_single_data_from_path(data_folder, all_info[i][j]['file_path'])
                I = data['current']
                V = data['voltage']
                draw_single_DC_IV(V, I, fig)
                plt.savefig(local_graph_folder + '\\' + str(j) + '.png', bbox_inches = 'tight', dpi = 200)
                plt.clf()
                plt.close()
                print(i, j)
                

In [3]:
draw_all_DC_IV_from_data_folder('7889')

E11 1
F12 1
F12 2
F12 3
F12 4
F12 5
F12 6
F12 7
F12 8
F12 9
F12 10
F12 11
F12 12
F12 13
F12 14
F12 15
F12 16
F12 17
F12 18
F12 19
F12 20
F12 21
F12 22
F12 23
F12 24
F12 25
F12 26
F12 27
F12 28
F12 29
F12 30
F12 31
F12 32
F12 33
F12 34
F12 35
F12 36
F12 37
F12 38
F12 39
F12 40
F12 41
F12 42
F12 43
F12 44
F12 45
F12 46
F12 47
F12 48
F12 49
F12 50
F12 51
F12 52
F12 53
F12 54
F12 55
F12 56
F12 57
F12 58
F12 59
F12 60
F13 1
F13 2
F13 3
F14 1
F14 2
F14 3
F14 4
F14 5
F14 6
F14 7
F14 8
F14 9
F14 10
F14 11
F14 12
F14 13
F14 14
F14 15
F14 16
F14 17
F14 18
F14 19
F14 20
F14 21
F14 22
F14 23
F14 24
F14 25
J13 1
J13 2
J13 3
J13 4
J13 5
J13 6
J13 7
J13 8
J13 9
J13 10
J13 11
J13 12
J13 13
J13 14
J13 15
J13 16
J13 17
J13 18
J13 19
J13 20
J14 1
J14 2
J14 3
J14 4
J14 5
J14 6
J14 7
J14 8
J14 9
J14 10
J14 11
J14 12
J14 13
J14 14
J14 15
J14 16
J14 17
J14 18
J14 19
J14 20
J14 21
J14 22
J14 23
J14 24
J14 25
J14 26
test 1
test 2
test 3
test 4
test 5
